In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql import * #for window

spark = SparkSession\
.builder\
.master("yarn")\
.appName("bucketing")\
.enableHiveSupport()\
.config("spark.sql.warehouse.dir","/user/itv009490/warehouse")\
.getOrCreate()

spark

In [5]:
df2 = spark.read.format("csv").option("inferSchema",True).\
load("/public/trendytech/retail_db/customers/part-00000")

In [6]:
df2 = df2.toDF("customer_id","firstname","lastname","email","password","street","city","state","zipcode")

In [7]:
df2.show()

+-----------+-----------+---------+---------+---------+--------------------+-------------+-----+-------+
|customer_id|  firstname| lastname|    email| password|              street|         city|state|zipcode|
+-----------+-----------+---------+---------+---------+--------------------+-------------+-----+-------+
|          1|    Richard|Hernandez|XXXXXXXXX|XXXXXXXXX|  6303 Heather Plaza|  Brownsville|   TX|  78521|
|          2|       Mary|  Barrett|XXXXXXXXX|XXXXXXXXX|9526 Noble Embers...|    Littleton|   CO|  80126|
|          3|        Ann|    Smith|XXXXXXXXX|XXXXXXXXX|3422 Blue Pioneer...|       Caguas|   PR|    725|
|          4|       Mary|    Jones|XXXXXXXXX|XXXXXXXXX|  8324 Little Common|   San Marcos|   CA|  92069|
|          5|     Robert|   Hudson|XXXXXXXXX|XXXXXXXXX|10 Crystal River ...|       Caguas|   PR|    725|
|          6|       Mary|    Smith|XXXXXXXXX|XXXXXXXXX|3151 Sleepy Quail...|      Passaic|   NJ|   7055|
|          7|    Melissa|   Wilcox|XXXXXXXXX|XXXXXXXXX|

In [8]:
spark.sql("""create database itv009490_bucketingdb""")

""


In [10]:
#2 levels pf partitioning

df2.write.format("parquet").\
mode("overwrite").\
bucketBy(4,"customer_id").\
saveAsTable("itv009490_bucketingdb.customers_new")

In [11]:
!hdfs dfs -ls /user/itv009490/warehouse

Found 6 items
drwxr-xr-x   - itv009490 supergroup          0 2023-10-29 05:48 /user/itv009490/warehouse/009490_retail.db
drwxr-xr-x   - itv009490 supergroup          0 2023-11-26 13:19 /user/itv009490/warehouse/assignment_7.db
drwxr-xr-x   - itv009490 supergroup          0 2023-11-26 13:41 /user/itv009490/warehouse/itv009490_assingment_ext.db
drwxr-xr-x   - itv009490 supergroup          0 2023-12-03 14:21 /user/itv009490/warehouse/itv009490_bucketingdb.db
drwxr-xr-x   - itv009490 supergroup          0 2023-11-26 11:54 /user/itv009490/warehouse/itv009490_caching_demo_ext.db
drwxr-xr-x   - itv009490 supergroup          0 2023-11-26 12:29 /user/itv009490/warehouse/itv009490_cachingdemo_db.db


In [12]:
!hdfs dfs -ls /user/itv009490/warehouse/itv009490_bucketingdb.db

Found 1 items
drwxr-xr-x   - itv009490 supergroup          0 2023-12-03 14:21 /user/itv009490/warehouse/itv009490_bucketingdb.db/customers_new


In [13]:
!hdfs dfs -ls /user/itv009490/warehouse/itv009490_bucketingdb.db/customers_new

Found 5 items
-rw-r--r--   3 itv009490 supergroup          0 2023-12-03 14:21 /user/itv009490/warehouse/itv009490_bucketingdb.db/customers_new/_SUCCESS
-rw-r--r--   3 itv009490 supergroup      85911 2023-12-03 14:21 /user/itv009490/warehouse/itv009490_bucketingdb.db/customers_new/part-00000-eeac2e44-4e78-4bfd-9cd4-3c5f921a2b34_00000.c000.snappy.parquet
-rw-r--r--   3 itv009490 supergroup      86808 2023-12-03 14:21 /user/itv009490/warehouse/itv009490_bucketingdb.db/customers_new/part-00000-eeac2e44-4e78-4bfd-9cd4-3c5f921a2b34_00001.c000.snappy.parquet
-rw-r--r--   3 itv009490 supergroup      86911 2023-12-03 14:21 /user/itv009490/warehouse/itv009490_bucketingdb.db/customers_new/part-00000-eeac2e44-4e78-4bfd-9cd4-3c5f921a2b34_00002.c000.snappy.parquet
-rw-r--r--   3 itv009490 supergroup      85587 2023-12-03 14:21 /user/itv009490/warehouse/itv009490_bucketingdb.db/customers_new/part-00000-eeac2e44-4e78-4bfd-9cd4-3c5f921a2b34_00003.c000.snappy.parquet


In [ ]:
df3 = spark.read.format("parquet").load("/user/itv009490/warehouse/itv009490_bucketingdb.db/customers_new")

In [16]:
spark.sql("""select count(*) from itv009490_bucketingdb.customers_new where state = 'TX'""").show()

+--------+
|count(1)|
+--------+
|     635|
+--------+



In [17]:
spark.sql("""select count(*) from itv009490_bucketingdb.customers_new where customer_id = 10""").show()

+--------+
|count(1)|
+--------+
|       1|
+--------+

